Once we have downloaded multiple bands, we need to ensure the all bands are consistently at a 10-meter resolution, which is the highest resolution.

* **Bands 2, 3, 4, and 8**: These have a native resolution of 10 meters.
* **Bands 11 and 12**: These typically have a native resolution of 20 meters.

Therefore, we need to resample the 20-meter bands to 10-meter resolution.

# Step 1: Open the Dataset (TIFF file)

Our multi-band dataset has all the bands stacked in a single file. So, we'll need to read each band, check its resolution, and resample if necessary.

# Step 2: Resample Bands


In [31]:
import rasterio
from rasterio.enums import Resampling


In [33]:
# Path to original TIFF file
all_bands_tiff_path = 'Data\84adf81eb4ae349dc1f28833fabdf54b/response.tiff'

In [37]:
# Open the source dataset
with rasterio.open(all_bands_tiff_path) as src:
    # Read in each band, some might already be at the desired resolution
    band2 = src.read(2)  
    band3 = src.read(3)
    band4 = src.read(4)
    band8 = src.read(8)
    band11 = src.read(12)
    band12 = src.read(13) # The last band

    # Get metadata and update it for the output file
    meta = src.meta.copy()
    transform = src.transform
    new_width = src.width * 2  # Doubling width for 10m resolution
    new_height = src.height * 2  # Doubling height for 10m resolution

    # Update meta for new file
    meta.update({
        "driver": "GTiff",
        "height": new_height,
        "width": new_width,
        "transform": transform * transform.scale((src.width / new_width), (src.height / new_height))
    })

    # Resample bands 11 and 12 from 20m to 10m
    band11_resampled = src.read(12, out_shape=(new_height, new_width), resampling=Resampling.bilinear)
    band12_resampled = src.read(13, out_shape=(new_height, new_width), resampling=Resampling.bilinear)


* **Transform Scaling**: This adjusts the geographic transform of the image to account for the change in resolution, ensuring that each pixel still maps accurately to geographic space.

* **Resampling**: Bilinear interpolation is used for resampling because it provides a good balance between computational efficiency and image quality for continuous data. This method interpolates pixel values to create new values for the higher resolution output.

* **Metadata Updating**: Metadata is crucial to maintain geospatial referencing information, ensuring that the output file is usable in GIS applications.

# Step 3: Write the New File with All Bands
Writing all bands into a new TIFF file while ensuring all are at the 10m resolution.

In [43]:
with rasterio.open('Data/resampled_output.tif', 'w', **meta) as dst:
    dst.write(band2, 1)  # Write previously 10m bands as they are
    dst.write(band3, 2)
    dst.write(band4, 3)
    dst.write(band8, 4)
    dst.write(band11_resampled, 5)  # Write resampled 20m bands
    dst.write(band12_resampled, 6)
